In [1]:
import os
os.environ["SPS_HOME"] = "/Users/fpetri/packages/fsps" 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import lbg_forecast.sps as sps
import lbg_forecast.sfh as sfh
import lbg_forecast.priors as pr
import lbg_forecast.hyperparams as hyp
import lbg_forecast.population_model as pop
import lbg_forecast.zhistory as zh
import lbg_forecast.priors as pr
from astropy.cosmology import WMAP9 as cosmo
from scipy.stats import truncnorm

In [3]:
sps_model = sps.initialise_sps_model(neb_em=False)

In [4]:
redshift_mass_prior_parameters = pr.setup_redshift_and_mass_priors(z_max=15)
sps_parameters = pop.generate_sps_parameters(100, redshift_mass_prior_parameters)

In [45]:
def sample_prior_parameters(nsamples, bounds, sigma_bounds):
    """Sample prior parameters (hyperparameters) for
    SPS model. 

    :param nsamples:
        Number of samples

    :param bounds:
        (no. priors, 2) shape array with minimum and maximum
        mean for gaussian distribution for a given prior in 
        each row. The last rows are for uniform priors, where
        each row is then the minimum and maximum of that uniform
        distribution. The number of gaussian priors is determined
        by the length of the sigma_bounds parameter.

    :param sigma_bounds:
        (no. gaussian priors, 2) shape array with minimum and maximum
        standard deviation for gaussian priors. len(sigma_bounds) gives
        number of gaussian priors. So for any rows of parameter bounds
        with index greater then len(sigma_bounds) will be treated as
        uniform priors

    :returns prior_parameters:
        (nsamples, len(bounds)) shape array of hyperparameters. Each row is a
        different reaslisation of the hyperparameters.
        Number of rows = nsamples. Columns are parameters,
        in order given by (mu1, sig1, mu2, sig2, ... ) up
        to uniform prior parameters which then go as
        (min1, max1, min2, max2 ... )
    
    """
    n_gaussian_priors = len(sigma_bounds)
    n_priors = len(bounds)
    n_prior_parameters = int(2*n_priors)
    prior_parameters = np.empty((nsamples, n_prior_parameters))

    indx = 0
    #loop over gaussian priors
    while(indx < n_gaussian_priors):

        parameter_bounds = bounds[indx, :]
        parameter_sigma_bounds = sigma_bounds[indx, :]
        prior_parameters[:, int(2*indx)] = np.random.uniform(parameter_bounds[0], parameter_bounds[1], (nsamples,))
        prior_parameters[:, int(2*indx+1)] = np.random.uniform(parameter_sigma_bounds[0], parameter_sigma_bounds[1], (nsamples,)) 

        indx+=1

    #loop over uniform priors
    while(indx < n_priors):

        parameter_bounds = bounds[indx, :]
        prior_parameters[:, int(2*indx)] = np.ones((nsamples,))*parameter_bounds[0]
        prior_parameters[:, int(2*indx+1)] = np.ones((nsamples,))*parameter_bounds[1]

        indx+=1

    return prior_parameters

In [39]:
hyperparameter_mu_bounds = np.array([[-2.5, 0.5],  #logzsol
                                  [0.0, 2.0],      #dust1
                                  [0.0, 4.0],      #dust2
                                  [-1.0, 0.4],     #dust_index
                                  [1.0, 1.0],      #igm_factor 
                                  [-4.0, -1.0],    #gas_log_u
                                  [-5.0, 1.0],     #log10(fagn)
                                  [5, 150],        #agn_tau
                                  [2, 10],         #nu
                                  [0.1, 1.0]       #sig
])

hyperparameter_sigma_max = np.array([[0.01, 3.0], #logzsol
                                     [0.01, 2.0], #dust1
                                     [0.01, 4.0], #dust2
                                     [0.01, 0.5], #dust_index
                                     [0.01, 2.0], #igm_factor 
                                     [0.01, 3.0], #gas_log_u
                                     [0.01, 6.0], #log10(fagn)
                                     [0.1, 145]   #agn_tau
])

In [47]:
sample_prior_parameters(100, hyperparameter_mu_bounds, hyperparameter_sigma_max)

array([[-0.32138746,  2.28123203,  1.33735379, ..., 10.        ,
         0.1       ,  1.        ],
       [-0.40439016,  1.88095734,  0.52353616, ..., 10.        ,
         0.1       ,  1.        ],
       [-1.89068906,  1.82096914,  0.05930904, ..., 10.        ,
         0.1       ,  1.        ],
       ...,
       [ 0.01815251,  1.19540877,  1.61703978, ..., 10.        ,
         0.1       ,  1.        ],
       [-0.57612206,  2.78963354,  0.07493364, ..., 10.        ,
         0.1       ,  1.        ],
       [-1.89104721,  0.29582568,  0.25226641, ..., 10.        ,
         0.1       ,  1.        ]])